In [ ]:

import pickle
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp

In [ ]:
with open("alldata.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))
print(data.keys())

name_list = data['name_list']
industry_list = data['industry_list']
index_list = data['index_list']
cusip_list = data['cusip_list']
universe = data['universe']

In [ ]:
universe

In [ ]:
rd.open_session()

ric = "ABI.BR"
start_date = "2019-09-01"
end_date = "2025-08-01"

price_df = rd.get_history(
    universe=ric,
    start=start_date,
    end=end_date,
    interval="daily"
)

df = pd.DataFrame({
    'price': price_df['TRDPRC_1'],
    'volume_shares': price_df['ACVOL_UNS'],
    'bid': price_df['BID'],
    'ask': price_df['ASK']
})

fundamental_response = rd.get_data(
    universe=ric,
    fields=[
        "TR.TotalReturn.Date",
        "TR.TotalReturn",           # Total Return Index
        "TR.PriceToBook",           # Price to Book (Market-to-Book)
        "TR.CompanyMarketCap"       # Market Cap
    ],
    parameters={
        "SDate": start_date,
        "EDate": end_date,
        "Frq": "D",
        "Curn": "EUR"
    }
)

print("\nFundamental data response:")
print(fundamental_response.head(10))
print(f"\nColumns: {fundamental_response.columns.tolist()}")

if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
    fund_df = fundamental_response.copy()
    
    if 'Date' in fund_df.columns:
        fund_df['Date'] = pd.to_datetime(fund_df['Date'])
        fund_df.set_index('Date', inplace=True)
        
        if 'Instrument' in fund_df.columns:
            fund_df.drop('Instrument', axis=1, inplace=True)
        
        print(f"\nFundamental data with Date index:")
        print(fund_df.head(10))
        
        df = df.join(fund_df, how='left')
    else:
        print("\nWARNING: No Date column found in fundamental data!")
        print("Available columns:", fund_df.columns.tolist())

column_mapping = {
    'Total Return': 'tri',
    'Price To Book Value': 'mtbv',
    'Company Market Cap': 'cap'
}

for old_name, new_name in column_mapping.items():
    if old_name in df.columns:
        df.rename(columns={old_name: new_name}, inplace=True)

df['volume'] = df['volume_shares'] * df['price']
df.drop('volume_shares', axis=1, inplace=True)

if 'mtbv' in df.columns:
    df['mtbv'] = df['mtbv'].ffill()
if 'cap' in df.columns:
    df['cap'] = df['cap'].ffill()

desired_order = ['price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
existing_cols = [col for col in desired_order if col in df.columns]
df = df[existing_cols]

print("\n" + "="*80)
print("FINAL DATASET")
print("="*80)
print(df.head(15))
print(f"\nShape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nNull counts:\n{df.isnull().sum()}")
print(f"\nSample with non-null data:")
print(df[df['tri'].notna()].head(10))

df.to_csv("ABI_BR_Data.csv", index=True)
print(f"\nData saved to ABI_BR_Data.csv")

rd.close_session()

In [ ]:
import time

rd.open_session()

unique_rics = universe['RIC'].unique()
print(f"Total rows in universe: {len(universe)}")
print(f"Unique RICs to process: {len(unique_rics)}")

start_date = "2019-09-01"
end_date = "2025-08-01"

all_data = []
failed_rics = []

total_rics = len(unique_rics)
print(f"\nProcessing {total_rics} unique stocks...")

for idx, ric in enumerate(unique_rics, 1):
    print(f"[{idx}/{total_rics}] Processing {ric}...", end=' ')
    
    try:
        # --- Fetch historical PRICING data ---
        price_df = rd.get_history(
            universe=ric,
            start=start_date,
            end=end_date,
            interval="daily"
        )
        
        # Extract only what we need
        df = pd.DataFrame({
            'RIC': ric,  # Add RIC column
            'price': price_df['TRDPRC_1'],
            'volume_shares': price_df['ACVOL_UNS'],
            'bid': price_df['BID'],
            'ask': price_df['ASK']
        })
        
        # --- Fetch FUNDAMENTAL data WITH DATES ---
        fundamental_response = rd.get_data(
            universe=ric,
            fields=[
                "TR.TotalReturn.Date",
                "TR.TotalReturn",
                "TR.PriceToBook",
                "TR.CompanyMarketCap"
            ],
            parameters={
                "SDate": start_date,
                "EDate": end_date,
                "Frq": "D",
                "Curn": "EUR"
            }
        )
        
        # Process fundamental data
        if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
            fund_df = fundamental_response.copy()
            
            if 'Date' in fund_df.columns:
                fund_df['Date'] = pd.to_datetime(fund_df['Date'])
                fund_df.set_index('Date', inplace=True)
                
                if 'Instrument' in fund_df.columns:
                    fund_df.drop('Instrument', axis=1, inplace=True)
                
                # Merge with price data
                df = df.join(fund_df, how='left')
        
        # Rename columns
        column_mapping = {
            'Total Return': 'tri',
            'Price To Book Value': 'mtbv',
            'Company Market Cap': 'cap'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df.rename(columns={old_name: new_name}, inplace=True)
        
        # Compute volume in EUR
        df['volume'] = df['volume_shares'] * df['price']
        df.drop('volume_shares', axis=1, inplace=True)
        
        # Forward-fill mtbv and cap
        if 'mtbv' in df.columns:
            df['mtbv'] = df['mtbv'].ffill()
        if 'cap' in df.columns:
            df['cap'] = df['cap'].ffill()
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Reorder columns
        desired_order = ['RIC', 'Date', 'price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
        existing_cols = [col for col in desired_order if col in df.columns]
        df = df[existing_cols]
        
        # Append to list
        all_data.append(df)
        
        print(f"✓ {len(df)} rows")
        
        # Rate limiting - sleep briefly to avoid overwhelming the API
        if idx % 10 == 0:
            time.sleep(2)
        
    except Exception as e:
        print(f"✗ Failed: {e}")
        failed_rics.append(ric)
        continue

# --- 6. Combine all data ---
print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)
print(f"Successfully processed: {len(all_data)} stocks")
print(f"Failed: {len(failed_rics)} stocks")

if failed_rics:
    print(f"\nFailed RICs:")
    for ric in failed_rics:
        print(f"  - {ric}")

# --- 7. Create combined DataFrame ---
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print(f"\nCombined dataset shape: {combined_df.shape}")
    print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"Unique stocks: {combined_df['RIC'].nunique()}")
    
    # Display sample
    print("\nSample data:")
    print(combined_df.head(10))
    
    # Save to CSV
    combined_df.to_csv("all_stocks_data.csv", index=False)
    print(f"\n✓ Saved to all_stocks_data.csv")
    
    # Show null counts
    print("\nNull counts by column:")
    print(combined_df.isnull().sum())
else:
    print("\nNo data retrieved!")

# --- 8. Close session ---
rd.close_session()

print("\nDone!")

In [ ]:
missing = set(universe['RIC'].unique()) - set(pd.read_csv("all_stocks_data.csv")['RIC'].unique())
print(f"Missing: {len(missing)} RICs")
print(sorted(missing))

In [ ]:
df = pd.read_csv("all_stocks_data.csv")

myday = df[['Date']].drop_duplicates().sort_values('Date')
myday.to_csv("myday.csv", index=False)

In [ ]:
combined_df = pd.read_csv("all_stocks_data.csv")

print("Analyzing existing data for incomplete RICs...")
print("="*80)
print(f"Columns in CSV: {combined_df.columns.tolist()}")
print(f"Shape: {combined_df.shape}")

# --- Find RICs with ANY missing data ---
# Only check columns that actually exist
columns_to_check = ['price', 'tri', 'volume', 'cap', 'bid', 'ask']
existing_columns = [col for col in columns_to_check if col in combined_df.columns]

print(f"\nChecking these columns for missing data: {existing_columns}")

agg_dict = {col: lambda x: x.isnull().sum() for col in existing_columns}

rics_with_missing = combined_df.groupby('RIC').agg(agg_dict).reset_index()

# Calculate total missing per RIC
rics_with_missing['total_missing'] = rics_with_missing[existing_columns].sum(axis=1)

# Filter to only RICs with ANY missing data
incomplete_rics = rics_with_missing[rics_with_missing['total_missing'] > 0]['RIC'].tolist()

print(f"\nRICs with complete data: {len(rics_with_missing) - len(incomplete_rics)}")
print(f"RICs with missing data: {len(incomplete_rics)}")

if len(incomplete_rics) == 0:
    print("\nNo incomplete RICs found! All data is complete.")
else:
    print(f"\nIncomplete RICs: {incomplete_rics[:20]}")  # Show first 20
    if len(incomplete_rics) > 20:
        print(f"... and {len(incomplete_rics) - 20} more")
    
    # Show summary of what's missing
    print("\nMissing data breakdown for incomplete RICs:")
    print(rics_with_missing[rics_with_missing['total_missing'] > 0].sort_values('total_missing', ascending=False).head(20))

# --- Retry all incomplete RICs ---
print("\n" + "="*80)
print(f"RETRYING {len(incomplete_rics)} INCOMPLETE RICs")
print("="*80)

rd.open_session()

retry_data = []
retry_failed = {}

start_date = "2019-09-01"
end_date = "2025-08-01"

for idx, ric in enumerate(incomplete_rics, 1):
    print(f"[{idx}/{len(incomplete_rics)}] Processing {ric}...", end=' ')
    
    try:
        # --- Fetch historical PRICING data ---
        price_df = rd.get_history(
            universe=ric,
            start=start_date,
            end=end_date,
            interval="daily"
        )
        
        # Extract only what we need
        df = pd.DataFrame({
            'RIC': ric,
            'price': price_df['TRDPRC_1'] if 'TRDPRC_1' in price_df.columns else None,
            'volume_shares': price_df['ACVOL_UNS'] if 'ACVOL_UNS' in price_df.columns else None,
            'bid': price_df['BID'] if 'BID' in price_df.columns else None,
            'ask': price_df['ASK'] if 'ASK' in price_df.columns else None
        })
        
        # --- Fetch FUNDAMENTAL data WITH DATES ---
        try:
            fundamental_response = rd.get_data(
                universe=ric,
                fields=[
                    "TR.TotalReturn.Date",
                    "TR.TotalReturn",
                    "TR.PriceToBook",
                    "TR.CompanyMarketCap"
                ],
                parameters={
                    "SDate": start_date,
                    "EDate": end_date,
                    "Frq": "D",
                    "Curn": "EUR"
                }
            )
            
            # Process fundamental data
            if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
                fund_df = fundamental_response.copy()
                
                if 'Date' in fund_df.columns:
                    fund_df['Date'] = pd.to_datetime(fund_df['Date'])
                    fund_df.set_index('Date', inplace=True)
                    
                    if 'Instrument' in fund_df.columns:
                        fund_df.drop('Instrument', axis=1, inplace=True)
                    
                    # Merge with price data
                    df = df.join(fund_df, how='left')
        
        except Exception as fund_error:
            pass  # Continue even if fundamentals fail
        
        # Rename columns
        column_mapping = {
            'Total Return': 'tri',
            'Price To Book Value': 'mtbv',
            'Company Market Cap': 'cap'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df.rename(columns={old_name: new_name}, inplace=True)
        
        # Compute volume in EUR
        if 'volume_shares' in df.columns and 'price' in df.columns:
            df['volume'] = df['volume_shares'] * df['price']
            df.drop('volume_shares', axis=1, inplace=True)
        
        # Forward-fill mtbv and cap
        if 'mtbv' in df.columns:
            df['mtbv'] = df['mtbv'].ffill()
        if 'cap' in df.columns:
            df['cap'] = df['cap'].ffill()
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        if 'index' in df.columns:
            df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Reorder columns - only include columns that exist
        desired_order = ['RIC', 'Date', 'price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
        existing_cols = [col for col in desired_order if col in df.columns]
        df = df[existing_cols]
        
        # Append to list
        retry_data.append(df)
        
        print(f"✓ {len(df)} rows")
        
        # Rate limiting
        if idx % 10 == 0:
            time.sleep(2)
        
    except Exception as e:
        print(f"✗ Failed: {e}")
        retry_failed[ric] = str(e)
        continue

rd.close_session()

# --- Save retry results ---
print("\n" + "="*80)
print("RETRY COMPLETE")
print("="*80)

if retry_data:
    attempt2_df = pd.concat(retry_data, ignore_index=True)
    
    print(f"Successfully re-retrieved: {len(retry_data)} RICs")
    print(f"Failed on retry: {len(retry_failed)} RICs")
    print(f"Total rows in attempt2: {len(attempt2_df)}")
    print(f"Date range: {attempt2_df['Date'].min()} to {attempt2_df['Date'].max()}")
    
    # Check if data is still incomplete
    print("\nMissing data in attempt2:")
    print(attempt2_df.isnull().sum())
    
    # Save to CSV
    attempt2_df.to_csv("attempt2.csv", index=False)
    print(f"\n✓ Saved to attempt2.csv")
    
    # Show which RICs still have issues
    rics_still_incomplete = attempt2_df.groupby('RIC').apply(
        lambda x: x.isnull().sum().sum()
    )
    rics_still_incomplete = rics_still_incomplete[rics_still_incomplete > 0]
    
    if len(rics_still_incomplete) > 0:
        print(f"\nWarning: {len(rics_still_incomplete)} RICs still have missing data after retry:")
        print(rics_still_incomplete.sort_values(ascending=False).head(20))
else:
    print("\nNo data retrieved on retry!")

if retry_failed:
    print(f"\nFailed RICs:")
    for ric, error in list(retry_failed.items())[:10]:
        print(f"  {ric}: {error[:80]}")
    
    # Save failed list
    pd.DataFrame([
        {'RIC': ric, 'Error': error} 
        for ric, error in retry_failed.items()
    ]).to_csv("retry_failed_rics.csv", index=False)
    print(f"\n✓ Saved failed RICs to retry_failed_rics.csv")

print("\nDone!")

In [ ]:
pd.read_csv("attempt2.csv").isnull().sum()

In [ ]:
combined_df = pd.read_csv("all_stocks_data.csv")
universe_unique = universe['RIC'].unique()

missing_rics = set(universe_unique) - set(combined_df['RIC'].unique())
missing_rics = sorted(missing_rics)

print(f"Found {len(missing_rics)} missing RICs")
print(f"Missing RICs: {missing_rics[:10]}...")  # Show first 10


pd.DataFrame({'RIC': missing_rics}).to_csv("completely_missing_rics.csv", index=False)


# --- Retry missing RICs with detailed error messages ---
print("\n" + "="*80)
print("RETRYING FAILED RICs")
print("="*80)

rd.open_session()

retry_data = []
still_failed = {}

for idx, ric in enumerate(missing_rics, 1):
    print(f"\n[{idx}/{len(missing_rics)}] Retrying {ric}...")
    
    try:
        # Try with a shorter date range first to see if data exists
        test_df = rd.get_history(
            universe=ric,
            start="2024-01-01",
            end="2024-12-31",
            interval="daily"
        )
        
        print(f"  Test fetch: {len(test_df)} rows returned")
        print(f"  Available fields: {test_df.columns.tolist()}")
        
        # If test works, fetch full range
        price_df = rd.get_history(
            universe=ric,
            start="2019-09-01",
            end="2025-08-01",
            interval="daily"
        )
        
        df = pd.DataFrame({
            'RIC': ric,
            'price': price_df['TRDPRC_1'] if 'TRDPRC_1' in price_df.columns else None,
            'volume_shares': price_df['ACVOL_UNS'] if 'ACVOL_UNS' in price_df.columns else None,
            'bid': price_df['BID'] if 'BID' in price_df.columns else None,
            'ask': price_df['ASK'] if 'ASK' in price_df.columns else None
        })
        
        # Try fundamentals
        try:
            fundamental_response = rd.get_data(
                universe=ric,
                fields=[
                    "TR.TotalReturn.Date",
                    "TR.TotalReturn",
                    "TR.PriceToBook",
                    "TR.CompanyMarketCap"
                ],
                parameters={
                    "SDate": "2019-09-01",
                    "EDate": "2025-08-01",
                    "Frq": "D",
                    "Curn": "EUR"
                }
            )
            
            if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
                fund_df = fundamental_response.copy()
                
                if 'Date' in fund_df.columns:
                    fund_df['Date'] = pd.to_datetime(fund_df['Date'])
                    fund_df.set_index('Date', inplace=True)
                    
                    if 'Instrument' in fund_df.columns:
                        fund_df.drop('Instrument', axis=1, inplace=True)
                    
                    df = df.join(fund_df, how='left')
        except Exception as fund_error:
            print(f"  Warning: Fundamentals failed: {fund_error}")
        
        # Rename columns
        column_mapping = {
            'Total Return': 'tri',
            'Price To Book Value': 'mtbv',
            'Company Market Cap': 'cap'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df.rename(columns={old_name: new_name}, inplace=True)
        
        # Compute volume in EUR
        if 'volume_shares' in df.columns and 'price' in df.columns:
            df['volume'] = df['volume_shares'] * df['price']
            df.drop('volume_shares', axis=1, inplace=True)
        
        # Forward-fill
        if 'mtbv' in df.columns:
            df['mtbv'] = df['mtbv'].ffill()
        if 'cap' in df.columns:
            df['cap'] = df['cap'].ffill()
        
        # Reset index
        df.reset_index(inplace=True)
        if 'index' in df.columns:
            df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Reorder columns
        desired_order = ['RIC', 'Date', 'price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
        existing_cols = [col for col in desired_order if col in df.columns]
        df = df[existing_cols]
        
        retry_data.append(df)
        print(f"  ✓ Success: {len(df)} rows")
        
        if idx % 5 == 0:
            time.sleep(2)
        
    except Exception as e:
        error_msg = str(e)
        print(f"  ✗ Failed: {error_msg}")
        still_failed[ric] = error_msg
        continue

rd.close_session()

if retry_data:
    all_retry_df = pd.concat(retry_data, ignore_index=True)
    all_retry_df.to_csv("missing.csv", index=False)
    print(f"Saved {len(all_retry_df)} rows to missing.csv")
else:
    print("No data was successfully retrieved for missing RICs.")


In [ ]:
all_df = pd.read_csv("all_stocks_data.csv")
missing_df = pd.read_csv("missing.csv")

new_rows = missing_df[~missing_df['RIC'].isin(all_df['RIC'])]

combined_df = pd.concat([all_df, new_rows], ignore_index=True)

combined_df.to_csv("all_stocks_data.csv", index=False)

print(f"Added {len(new_rows)} new rows to all_stocks_data.csv")

In [ ]:

all_df = pd.read_csv("all_stocks_data.csv")
attempt2_df = pd.read_csv("attempt2.csv")

# Get the RICs present in attempt2.csv
ric_to_replace = attempt2_df['RIC'].unique()

# Keep only rows in all_df whose RIC is NOT in attempt2_df
all_df_filtered = all_df[~all_df['RIC'].isin(ric_to_replace)]

# Append the attempt2 data
combined_df = pd.concat([all_df_filtered, attempt2_df], ignore_index=True)

# Save back to all_stocks_data.csv
combined_df.to_csv("all_stocks_data.csv", index=False)

print(f"Replaced data for {len(ric_to_replace)} RICs with attempt2.csv")

In [50]:
pd.read_csv("all_stocks_data.csv").isnull().sum()

RIC            0
Date           0
price     332682
tri          376
volume    332768
cap         3050
bid          499
ask          570
dtype: int64